<h1>K Nearest Neighbors Model for Heart Disease </h1>

<b>Import Libraries </b>

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
from sklearn.preprocessing import StandardScaler #import libraries for preprocessing dataset
from sklearn.model_selection import train_test_split#import libraries for Create Train Test dataset
from sklearn.metrics import accuracy_score #import libraries for check Accuracy
from sklearn.metrics import confusion_matrix#import libraries for create matrix
from sklearn.neighbors import KNeighborsClassifier# import libraries for K Nearest neighbors model
from sklearn.metrics import classification_report #import libraries for classification

In [4]:
import eli5
from eli5.sklearn import PermutationImportance

In [5]:
import shap

In [6]:
import warnings
warnings.filterwarnings('ignore')

<b>Import CSV file</b>

In here I import heart.csv file as a dataset. It is a one of real-time dataset of the kaggle.

In [7]:
heartData = pd.read_csv('heart.csv')

In [8]:
heartData.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


<b>Rename columns in CSV file</b>

In [9]:
heartData = heartData.rename(columns={"cp": "ChestPain", "trestbps": "BloodPressure", "fbs": "BloodSugar", "ca": "Vessels", "chol": "Cholesterol"})

In [10]:
heartData.describe()

,age,sex,ChestPain,BloodPressure,Cholesterol,BloodSugar,restecg,thalach,exang,oldpeak,slope,Vessels,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


<b>Standardize features</b>

In [12]:
standardScaler = StandardScaler()
columns_to_scale = ['age', 'BloodPressure', 'Cholesterol', 'thalach', 'oldpeak']
heartData[columns_to_scale] = standardScaler.fit_transform(heartData[columns_to_scale])#data fit and transfroming

<b>Convert categorical variable into dummy/indicator variables</b>

In [13]:
heartData = pd.get_dummies(heartData, columns = ['sex', 'ChestPain', 'BloodSugar', 'restecg', 'exang', 'slope', 'thal'], drop_first=True)

<b>Features and target labels to be kept separate (healthy or sick)</b>

In [14]:
labels = heartData['target']
features = heartData.drop(['target'], axis = 1)

Create two sets of features and target labels: one for training and one for testing.

In [15]:
features_train , features_test, labels_train, labels_test = train_test_split(features, labels, test_size= 0.2, random_state=42)